In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('./nasdaq_d_2012_2022.csv')
df = df.pivot(index="date",columns="tic", values="adjcp")
df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')
tick = ['TSLA', 'NVDA', 'AMD', 'AVGO', 'ALGN', 'LRCX', 'FTNT', 
          'ODFL', 'AMAT','MSFT', 'MU', 'IDXX', 'KLAC', 'CDNS', 
          'MRVL', 'ASML', 'FB', 'CTAS','INTU', 'SNPS']
df = df.loc[:,tick]

In [4]:
#TODO:NOISE 随机扰动
class portOptim(object):
    def __init__(self,real_df,pred_df, dynamic_win = True, tran_cost = 0.001, rf = 0.06):
        self.df = real_df.iloc[1:,:]
        self.date = pred_df.index
        self.pct_dt = real_df.pct_change().iloc[1:,:]
        self.pred_df = pred_df
        self.day = 0
        self.max_day = pred_df.shape[0]
        self.covWin = 30
        self.data = self.df.iloc[self.day:self.day+self.covWin,:]
        self.init_action = self.action = np.array([0]*20)
        self.mu = 0
        self.cov = 0
        self.risk_re = 0.5
        self.dynamic_win = dynamic_win
        
        #portfolio
        self.p = lambda A: self.asset + np.append(-A.sum()-abs(A).sum()*self.tran_cost,A)

        #weight
        self.w = lambda A: (self.p(A) / np.sum(self.p(A)))

        #Penalties for handling fees
        self.pena = lambda A: abs(A).sum()/self.p(A).sum()
        
        #sharpe ratio
        self.SR = lambda A: (self.mu@self.w(A)-self.pena(A))/np.sqrt(self.w(A)@self.cov@self.w(A))
        self.op = lambda A: -self.SR(A)


        ones = np.ones(21)
        self.cons = ({'type': 'ineq', 'fun': lambda A: self.w(A)},
                {'type': 'eq', 'fun': lambda A: self.w(A) @ ones - 1})
        
        self.tran_cost = tran_cost
        self.rf = rf/365
        self.asset = self.weight = np.array([1/21]*21)
        self.scaler = MinMaxScaler(feature_range=(-30,-5))
        self.action_memory = []
        self.asset_memory = []
        self.tick = np.array(tick)
#         print(f'Day{self.day}, initial portfolio {self.asset} , total {round(self.asset.sum(),2)}')
        
    
    def get_mu(self):
        cov = np.array(self.data.cov())
        var = cov.diagonal().reshape(20,1)
        win_s = self.scaler.fit_transform(var).astype('int').flatten()
        pred_c = np.array(self.pred_df.iloc[self.day,:]/self.df.iloc[self.day+self.covWin-1,:])
        mu = []
        for i in range(len(win_s)):
            pct_c = self.data.iloc[win_s[i]:,i]
            mu.append(pct_c.mean())

        dynamic_mu = np.append(self.rf,np.array(mu*pred_c-1)/(win_s+1))
        normal_mu = np.append(self.rf,np.array(self.data.mean()+pred_c/self.covWin))

#         print(f'Var\n{var.squeeze()}\n winSize\n {win_s}\n mu \n{dynamic_mu}')

        return dynamic_mu if self.dynamic_win else normal_mu
        
    def optim(self):
        result = minimize(self.op, x0=(self.action), method='trust-constr', constraints=self.cons)
        return result.x
                      
    def step(self):
        if self.day>=self.max_day:
            print('Finished.')
            return True
        self.data = self.pct_dt.iloc[self.day:self.day+self.covWin,:]
        self.mu = self.get_mu()

        cov = self.data.cov()
        self.cov = np.block([[0,np.zeros(20)],[np.zeros([20,1]),cov]])

        #get the action for achive the SR maximum
        self.action = self.init_action
        self.action = self.optim().astype('float64')
        SR = self.SR(self.action)

        self.action_memory.append(self.action)
#         print(f'Action:\n {self.action}')
        real_c = np.array(self.df.iloc[self.day+self.covWin,:]/self.df.iloc[self.day+self.covWin-1,:])
#         print(f'Real return {real_c}')
        self.asset = self.p(self.action)*np.append(1+self.rf,real_c)
        self.asset_memory.append(self.asset)
#         print(f'Asset:\n {self.asset}')
        #calculate the real asset
        self.weight = self.asset/self.asset.sum()
        self.day+=1
        top_3_idx = self.weight[1:].argsort()[-3:][::-1]
        top_3 = self.tick[top_3_idx]
        print(f'Day{self.day}, {self.date[self.day-1]}, SR {SR}, risk free {round(self.weight[0],2)} TOP3 {top_3},################ total {round(self.asset.sum(),2)}')
        print('#######################################################')
        return False
        
    def perf(self):
        while(1):
            done = self.step()
            if done:
                break

In [5]:
lag = 0
df_pred = pd.read_csv("./prediction.csv", index_col=0).iloc[lag:,:]
df_real = df.loc['2019-11-15':,:].iloc[lag:,:]

## Sensitivity test

In [28]:
tran_cost_list = np.arange(0,0.003,0.0005)
rf_list = np.arange(0,0.121,0.02)
# tran_cost_list
rf_list

array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12])

In [ ]:
# senstive_result = {}
for i in [0]:
    for j in range(len(rf_list)):
        tran_cost = tran_cost_list[i]
        rf = rf_list[j]
        print(f'Model{(i+1)*(j+1)}\n trans_cost = {tran_cost}, rf{rf}')
        model = portOptim(df_real,df_pred,dynamic_win = False, tran_cost = tran_cost,rf = rf)
        model.perf()
        senstive_result[j] = model

Model1
 trans_cost = 0.0, rf0.0
Day1, 2020-01-02, SR 3.948843263201017, risk free 0.05 TOP3 ['AMD' 'MRVL' 'CDNS'],################ total 1.02
#######################################################
Day2, 2020-01-03, SR 3.652450730372774, risk free 0.05 TOP3 ['AMD' 'TSLA' 'FTNT'],################ total 1.01
#######################################################
Day3, 2020-01-06, SR 3.4437410748340924, risk free 0.05 TOP3 ['TSLA' 'FTNT' 'AMD'],################ total 1.01
#######################################################
Day4, 2020-01-07, SR 3.611936250174482, risk free 0.05 TOP3 ['TSLA' 'MU' 'FTNT'],################ total 1.02
#######################################################
Day5, 2020-01-08, SR 3.4231435068963054, risk free 0.05 TOP3 ['TSLA' 'FTNT' 'MU'],################ total 1.03
#######################################################
Day6, 2020-01-09, SR 3.9488117013276245, risk free 0.04 TOP3 ['TSLA' 'FTNT' 'SNPS'],################ total 1.04
##########################

Day50, 2020-03-13, SR 0.629506653765391, risk free 0.06 TOP3 ['TSLA' 'CTAS' 'IDXX'],################ total 0.91
#######################################################
Day51, 2020-03-16, SR 0.3408687680156011, risk free 0.05 TOP3 ['TSLA' 'IDXX' 'CTAS'],################ total 0.78
#######################################################
Day52, 2020-03-17, SR 0.519203310976875, risk free 0.05 TOP3 ['TSLA' 'MSFT' 'NVDA'],################ total 0.82
#######################################################
Day53, 2020-03-18, SR 0.48913469000832804, risk free 0.05 TOP3 ['MSFT' 'ODFL' 'NVDA'],################ total 0.75
#######################################################
Day54, 2020-03-19, SR 0.36375951103390736, risk free 0.05 TOP3 ['TSLA' 'MSFT' 'NVDA'],################ total 0.77
#######################################################
Day55, 2020-03-20, SR 0.39834787406306654, risk free 0.05 TOP3 ['TSLA' 'NVDA' 'MSFT'],################ total 0.76
#########################################

Day99, 2020-05-22, SR 0.4905839592804263, risk free 0.03 TOP3 ['TSLA' 'ALGN' 'NVDA'],################ total 1.13
#######################################################
Day100, 2020-05-26, SR 1.3988687950054461, risk free 0.02 TOP3 ['TSLA' 'ALGN' 'NVDA'],################ total 1.14
#######################################################
Day101, 2020-05-27, SR 1.5620079923639334, risk free 0.02 TOP3 ['TSLA' 'ALGN' 'MU'],################ total 1.14
#######################################################
Day102, 2020-05-28, SR 1.5242873917917967, risk free 0.02 TOP3 ['TSLA' 'ALGN' 'IDXX'],################ total 1.14
#######################################################
Day103, 2020-05-29, SR 1.5639273789402857, risk free 0.02 TOP3 ['TSLA' 'ALGN' 'MRVL'],################ total 1.16
#######################################################
Day104, 2020-06-01, SR 0.7754278812138863, risk free 0.02 TOP3 ['TSLA' 'ALGN' 'MRVL'],################ total 1.17
#######################################

Day148, 2020-08-03, SR 2.3696952610923825, risk free 0.04 TOP3 ['TSLA' 'ODFL' 'IDXX'],################ total 1.42
#######################################################
Day149, 2020-08-04, SR 1.8516289272131747, risk free 0.03 TOP3 ['TSLA' 'AMD' 'ODFL'],################ total 1.43
#######################################################
Day150, 2020-08-05, SR 1.9244163185230463, risk free 0.04 TOP3 ['TSLA' 'AMD' 'ODFL'],################ total 1.43
#######################################################
Day151, 2020-08-06, SR 2.2984824326060633, risk free 0.04 TOP3 ['TSLA' 'AMD' 'ODFL'],################ total 1.43
#######################################################
Day152, 2020-08-07, SR 2.611499066773742, risk free 0.04 TOP3 ['TSLA' 'AMD' 'ODFL'],################ total 1.42
#######################################################
Day153, 2020-08-10, SR 0.039674410527979424, risk free 0.0 TOP3 ['TSLA' 'AMD' 'ODFL'],################ total 1.41
#########################################

Day197, 2020-10-12, SR 0.4551893275816611, risk free 0.0 TOP3 ['TSLA' 'ALGN' 'NVDA'],################ total 1.58
#######################################################
Day198, 2020-10-13, SR 1.0503034612339885, risk free 0.0 TOP3 ['TSLA' 'ALGN' 'NVDA'],################ total 1.59
#######################################################
Day199, 2020-10-14, SR -1.3988183062279946, risk free 0.0 TOP3 ['TSLA' 'ALGN' 'NVDA'],################ total 1.59
#######################################################
Day200, 2020-10-15, SR 1.3600953690163728, risk free 0.0 TOP3 ['TSLA' 'ALGN' 'NVDA'],################ total 1.58
#######################################################
Day201, 2020-10-16, SR 1.3877822134827569, risk free 0.0 TOP3 ['TSLA' 'ALGN' 'NVDA'],################ total 1.57
#######################################################
Day202, 2020-10-19, SR 1.5257469342056051, risk free 0.0 TOP3 ['TSLA' 'ALGN' 'NVDA'],################ total 1.54
#########################################

...

Day431, 2021-09-16, SR 1.6532483614977163, risk free 0.19 TOP3 ['TSLA' 'MU' 'CDNS'],################ total 3.83
#######################################################
Day432, 2021-09-17, SR 1.1620029317216762, risk free 0.19 TOP3 ['TSLA' 'MU' 'MRVL'],################ total 3.83
#######################################################
Day433, 2021-09-20, SR 1.3594323022344772, risk free 0.2 TOP3 ['TSLA' 'MU' 'MRVL'],################ total 3.72
#######################################################
Day434, 2021-09-21, SR 1.1755225047162445, risk free 0.2 TOP3 ['TSLA' 'MU' 'KLAC'],################ total 3.75
#######################################################
Day435, 2021-09-22, SR 1.2793016888315796, risk free 0.19 TOP3 ['TSLA' 'MU' 'CTAS'],################ total 3.81
#######################################################
Day436, 2021-09-23, SR 1.2786131842499044, risk free 0.2 TOP3 ['TSLA' 'MU' 'IDXX'],################ total 3.81
###################################################

Day480, 2021-11-24, SR 0.33618315608114147, risk free 0.31 TOP3 ['TSLA' 'AVGO' 'AMD'],################ total 5.27
#######################################################
Day481, 2021-11-26, SR 0.9884988563628333, risk free 0.31 TOP3 ['TSLA' 'AVGO' 'AMD'],################ total 5.16
#######################################################
Day482, 2021-11-29, SR 0.7520104593698922, risk free 0.3 TOP3 ['TSLA' 'AVGO' 'AMD'],################ total 5.33
#######################################################
Day483, 2021-11-30, SR -8.119239282942745, risk free 0.05 TOP3 ['TSLA' 'AVGO' 'AMAT'],################ total 5.33
#######################################################
Day484, 2021-12-01, SR -0.33831340106552865, risk free 0.03 TOP3 ['TSLA' 'AVGO' 'AMAT'],################ total 5.15
#######################################################
Day485, 2021-12-02, SR -0.045927512043034524, risk free 0.03 TOP3 ['TSLA' 'AVGO' 'AMAT'],################ total 5.11
##################################

Day24, 2020-02-05, SR 2.245247830860553, risk free 0.06 TOP3 ['TSLA' 'MSFT' 'LRCX'],################ total 1.09
#######################################################
Day25, 2020-02-06, SR 2.3112427612269757, risk free 0.06 TOP3 ['TSLA' 'MSFT' 'ODFL'],################ total 1.1
#######################################################
Day26, 2020-02-07, SR -1.3256847929983435, risk free 0.0 TOP3 ['TSLA' 'LRCX' 'CDNS'],################ total 1.08
#######################################################
Day27, 2020-02-10, SR 1.7422648400543896, risk free 0.01 TOP3 ['TSLA' 'LRCX' 'NVDA'],################ total 1.1
#######################################################
Day28, 2020-02-11, SR 1.772365902857896, risk free 0.02 TOP3 ['TSLA' 'LRCX' 'NVDA'],################ total 1.11
#######################################################
Day29, 2020-02-12, SR 2.3791455104667443, risk free 0.02 TOP3 ['TSLA' 'LRCX' 'NVDA'],################ total 1.12
##############################################

Day73, 2020-04-16, SR 0.56859327487546, risk free 0.01 TOP3 ['TSLA' 'NVDA' 'LRCX'],################ total 1.01
#######################################################
Day74, 2020-04-17, SR 0.1872842407131649, risk free 0.01 TOP3 ['TSLA' 'NVDA' 'FTNT'],################ total 1.03
#######################################################
Day75, 2020-04-20, SR -0.503808657590042, risk free 0.01 TOP3 ['TSLA' 'NVDA' 'AMD'],################ total 1.01
#######################################################
Day76, 2020-04-21, SR 0.5054782982894475, risk free 0.0 TOP3 ['TSLA' 'NVDA' 'AMD'],################ total 0.96
#######################################################
Day77, 2020-04-22, SR 0.3294468106871503, risk free 0.02 TOP3 ['NVDA' 'AMD' 'TSLA'],################ total 1.01
#######################################################
Day78, 2020-04-23, SR 0.6026450208893382, risk free 0.02 TOP3 ['NVDA' 'AMD' 'LRCX'],################ total 1.0
##################################################

Day122, 2020-06-25, SR 2.0670246512910384, risk free 0.04 TOP3 ['NVDA' 'TSLA' 'FTNT'],################ total 1.23
#######################################################
Day123, 2020-06-26, SR 0.3156333525770047, risk free 0.05 TOP3 ['NVDA' 'TSLA' 'FTNT'],################ total 1.21
#######################################################
Day124, 2020-06-29, SR 2.0831729349333044, risk free 0.05 TOP3 ['TSLA' 'NVDA' 'FTNT'],################ total 1.22
#######################################################
Day125, 2020-06-30, SR 1.6498963344750444, risk free 0.04 TOP3 ['TSLA' 'NVDA' 'FTNT'],################ total 1.26
#######################################################
